In [1]:
# 

In [2]:
import os
os.environ['OPENAI_API_KEY']='sk-111111111111111111111111111111111111111111111111'
os.environ['OPENAI_API_BASE']='http://127.0.0.1:5000/v1'

In [3]:
import requests
model_info_url = 'http://127.0.0.1:5000/v1/internal/model/info'
resp = requests.get(model_info_url)
model_name = resp.json()['model_name']

print(model_name)

WestLake-7B-v2-8.0bpw-h8-exl2


In [4]:
from guidance import models, gen, system, user, assistant

In [5]:
model = models.OpenAIChat("gpt-3.5",base_url=os.environ['OPENAI_API_BASE'])
modeln = models.OpenAI("gpt-3.5",base_url=os.environ['OPENAI_API_BASE'])

# Oobabooga Custom Class
# modeln = models.Oobabooga("gpt-3.5",base_url=os.environ['OPENAI_API_BASE'])
# model = models.OobaboogaChat("gpt-3.5-turbo",base_url=os.environ['OPENAI_API_BASE'])

In [6]:
# this is being chaches somewhere. Need to figure it out. 
# Cache is at ~/.cache/guidance/openai.tokens/cache.db

In [7]:
with system():
    lm = model + "You are a helpful assistant."
with user():
    lm += "Who won the last Kentucky derby and by how much?"
with assistant():
    lm + gen(max_tokens=10)

In [8]:
with system():
    lm = model + "You are a helpful assistant."
with user():
    lm += "What is the meaning of life?"
with assistant():
    lm += gen("response")

In [9]:
modeln + '''\
Q: Who won the last Kentucky derby and by how much?
A:''' + gen(stop="Q:")

# Simple templates

In [10]:
query = "Who won the last Kentucky derby and by how much?"
modeln + f'''\
Q: {query}
A: {gen(stop="Q:", temperature=0.1)}'''

## Capturing variables

In [11]:
query = "Who won the last Kentucky derby and by how much?"
lm = modeln + f'''\
Q: {query}
A: {gen(name="answer", stop="Q:", temperature=0.01)}'''

In [12]:
lm["answer"]

'2021 Kentucky Derby was won by Medina Spirit ridden by John Velazquez. Medina Spirit won by 3 1/2 lengths.'

# Function encapsulation

In [13]:
import guidance

In [14]:
@guidance
def qa_bot(lm, query):
    lm += f'''\
    Q: {query}
    A: {gen(name="answer", stop="Q:", temperature=0.01)}'''
    return lm

In [15]:
query = "Who won the last Kentucky derby and by how much?"
modeln + qa_bot(query) # note we don't pass the `lm` arg here  (that will get passed during execution when it gets added to the model)

# Selecting among alternatives

In [16]:
from guidance import select

In [17]:
if model_name.lower().startswith('qwen'):
    model_pre = "<|im_start|>systemYou are a helpful assistant.<|im_end|><|im_start|>user"
    model_pos = "<|im_end|><|im_start|>assistant"
else:
    model_pre = ""
    model_pos = ""

In [18]:
modeln + f'''\
{model_pre}Q: {query}
Now I will choose to either SEARCH the web or RESPOND.{model_pos}
Choice: {select(["SEARCH", "RESPOND"], name="choice")}'''

In [19]:
modeln + f'''\
{model_pre}Q: {query}
Now I will choose to either SEARCH the web or RESPOND.{model_pos}
Choice: {select(["SEARCH", "RESPOND"], name="choice")}'''

# Interleaved generation and control

In [20]:
@guidance
def qa_bot(lm, query):
    lm += f'''\
    {model_pre}
    Q: {query}
    Now I will choose to either SEARCH the web or RESPOND.{model_pos}
    Choice: {select(["SEARCH", "RESPOND"], name="choice")}
    '''
    if lm["choice"] == "SEARCH":
        lm += "A: I don't know, Google it!"
    else:
        lm += f'A: {gen(stop="Q:", name="answer")}'
    return lm

modeln + qa_bot(query)


## generating lists

In [21]:
lm = modeln + f'''\
{model_pre}
Q: {query}
Now I will choose to either SEARCH the web or RESPOND.{model_pos}
Choice: {select(["SEARCH", "RESPOND"], name="choice")}
'''
if lm["choice"] == "SEARCH":
    lm += "Here are 3 search queries:\n"
    for i in range(3):
        lm += f'''{i+1}. "{gen(stop='"', name="queries", temperature=1.0, list_append=True)}"\n'''

# chat

In [22]:
from guidance import system, user, assistant

with system():
    lm = model + "You are a helpful assistant."

with user():
    lm += "What is the meaning of life?"

with assistant():
    lm += gen("response", temperature=0.01)


In [23]:
#

In [24]:
# you can create and guide multi-turn conversations by using a series of role tags
@guidance
def experts(lm, query):
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""\
        I want a response to the following question:
        {query}
        Who are 3 world-class experts (past or present) who would be great at answering this?
        Please don't answer the question or comment on it yet."""

    with assistant():
        lm += gen(name='experts', max_tokens=300, temperature=0.01)
    
    with user():
        lm += f"""\
        Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
        In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
        If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
        Please start your answer with ANSWER:"""
    
    with assistant():
        lm += gen(name='answer', max_tokens=600, temperature=0.01)

    return lm
                   
model + experts(query='What is the meaning of life?')

# streaming

In [25]:
for part in modeln.stream() + qa_bot(query):
    print(part) # do something with the partially executed lm




Q: Who won the last Kentucky derby and by how much?
Now I will choose to either SEARCH the web or RESPOND.
Choice:

Q: Who won the last Kentucky derby and by how much?
Now I will choose to either SEARCH the web or RESPOND.
Choice: SEARCH
A: I don't know, Google it!


In [26]:
query

'Who won the last Kentucky derby and by how much?'